In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)

url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
    "X-RapidAPI-Key": "af196f0478mshd90862dc657e127p18d90bjsn598fe178eab7",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

def llamar_pelis(url, headers, year):
    datos_paginas = []
    base_url = "https://moviesdatabase.p.rapidapi.com"
    querystring = {"genre": "Drama", "startYear": year, "titleType": "movie", "endYear": year}
    while url:
        if not url.startswith('http'):
            url = base_url + url
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            json_pelis = response.json()
            datos_paginas.extend(json_pelis['results'])
            url = json_pelis.get('next')
            querystring = {}  # No se necesitan más parámetros ya que 'next' contiene la nueva URL con parámetros incluidos
        else:
            print("Error en la solicitud:", response.status_code, response.reason)
            return None
    return {"results": datos_paginas}

def limpiar_json(datos_json):
    peliculas = []
    for pelicula in datos_json['results']:
        tipo = pelicula['titleType']['text']
        nombre = pelicula['titleText']['text']
        year = pelicula['releaseYear']['year']
        mes = pelicula['releaseDate']['month'] if 'releaseDate' in pelicula and pelicula['releaseDate'] is not None else 'No consta'
        id_pelicula = pelicula['id']
        peliculas.append({
            'Tipo': tipo,
            'Nombre': nombre,
            'Año de Estreno': year,
            'Mes de Estreno': mes,
            'ID': id_pelicula
        })
    return pd.DataFrame(peliculas)

In [ ]:
# Proceso iterativo sobre el rango de años
todos_los_datos = []
for year in range(2010, 2025):  # desde 2010 hasta 2024 inclusive
    datos_anuales = llamar_pelis(url, headers, year)
    if datos_anuales:
        df_anual = limpiar_json(datos_anuales)
        todos_los_datos.append(df_anual)

In [ ]:
# Concatena todos los DataFrames anuales en uno solo
df_final = pd.concat(todos_los_datos, ignore_index=True)
print(df_final.head (50))

In [ ]:
# df_final.to_csv("peliculas_por_año.csv", index=False) # Para convertir el DataFrame a un CSV